In [1]:
import numpy as np
import pandas as pd
import datasets
import torch
import transformers
import typing
import requests
import tqdm
import pathlib
import json

import medrag.utils
import medrag.finetune

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
retriever = medrag.utils.RetrievalSystem(corpus_name="Textbooks", db_dir="./corpus")
finetune = medrag.utils.RetrievalSystem(retriever_name="./cache/medcpt-embedding-model", corpus_name="Textbooks", db_dir="./corpus")

benchmark_df = medrag.finetune.load_benchmark_dataset()

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

/home/user/mambaforge/envs/tensorml/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [4]:
def compare(rows):
  results = []
  for row in rows:
    orig_docs, orig_scores = retriever.retrieve(row['question'], k=1)
    finetune_docs, finetune_scores = finetune.retrieve(row['question'], k=1)
    results.append({
        'question': row['question'],
        'original': orig_docs[0]['content'],
        'finetune': finetune_docs[0]['content'],
        'original_score': orig_scores[0],
        'finetune_score': finetune_scores[0],
    })
  return pd.DataFrame(results)

# show full text in pandas display
pd.set_option('display.max_colwidth', None)
compare(benchmark_df.iloc[20:25].to_dict('records'))

,question,original,finetune,original_score,finetune_score
0,"A 77-year-old woman presents to the emergency room with the complaints of fever, malaise, and night sweats. She recently observed an enlargement of her axillary lymph nodes, which she examines on a weekly basis. She has a remote history of breast cancer in her 60s that was treated with radiation and chemotherapy. She also reports a history of extensive travel to Africa and a 30-pack-year history of smoking. On physical exam, several axillary lymph nodes are palpable with a large non-tender palpable mass in her right axilla measuring 10 x 8 cm. Fine-needle aspiration demonstrates what the pathologist describes as ""a centroblastic and immunoblastic cell presence, suspicious for non-Hodgkin’s lymphoma (NHL)–diffuse large B cell variant"". Which of the following risk factors is responsible for this patient’s condition?","(See Chap. 134) Hodgkin’s disease occurs mainly in the age range that coincides with child-bearing. However, Hodgkin’s disease is not more common in pregnant than nonpregnant women. Hodgkin’s disease is diagnosed in approximately 1 in 6000 pregnancies. It generally presents as a nontender lymph node swelling, most often in the left supraclavicular region. It may be accompanied by B symptoms (fever, night sweats, unexplained weight loss). Excisional biopsy is the preferred diagnostic procedure because fine-needle aspiration cannot reveal the architectural framework that is an essential component of Hodgkin’s disease diagnosis. The stage at presentation appears to be unaffected by pregnancy. Women diagnosed in the second and third trimester can be treated safely with combination chemotherapy, usually doxorubicin, bleomycin, vinblastine, and dacarbazine (ABVD). In general, the patient in the first trimester is asymptomatic, and a woman with a desired pregnancy can be followed until the","Thomas B. Nutman, Peter F. Weller",63.532391,62.649166
1,"A 3-month-old infant is brought to her pediatrician because she coughs and seems to have difficulty breathing while feeding. In addition, she seems to have less energy compared to other babies and appears listless throughout the day. She was born by cesarean section to a G1P1 woman with no prior medical history and had a normal APGAR score at birth. Her parents say that she has never been observed to turn blue. Physical exam reveals a high-pitched holosystolic murmur that is best heard at the lower left sternal border. The most likely cause of this patient's symptoms is associated with which of the following abnormalities?","Not all continuous murmurs are pathologic. A continuous venous hum can be heard in healthy children and young adults, especially during pregnancy; it is best appreciated in the right supraclavicular fossa and can be obliterated by pressure over the right internal jugular vein or by having the patient turn his or her head toward the examiner. The continuous mammary souffle of pregnancy is created by enhanced arterial flow through engorged breasts and usually appears during the late third trimester or early puerperium. The murmur is louder in systole. Firm pressure with the diaphragm of the stethoscope can eliminate the diastolic portion of the murmur. (Table 51e-2; see Table 267-1) Careful attention to the behavior of heart murmurs during simple maneuvers that alter cardiac hemodynamics can provide important clues to their cause and significance. Respiration Auscultation should be performed during quiet respiration or with a modest increase in inspiratory effort, as more forceful","Thomas B. Nutman, Peter F. Weller",64.494247,62.664444
2,"A 30-year-old African American woman comes to the physician for the evaluation of a dry cough and chest discomfort for the past 3 days. During this period, the patient has had headaches, muscle aches, joint pain, fever, and chills. Ten days ago, she was hiking with her family in Mississippi. The patient has asthma that is treated with an albuterol inhaler. Her m